In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [5]:
# Example file for testing / developing a quick understanding
filename = "../datasets/MBTA-Bus-Arrival-Departure-Times_2024-12.csv"
df = pd.read_csv(filename, low_memory=False)

In [6]:
# Calculate delay for each line
df['time_difference'] = pd.to_datetime(df['actual']) - pd.to_datetime(df['scheduled'])
df['delay_seconds'] = df['time_difference'].dt.total_seconds()
df['delay_headway'] = df['headway'] - df['scheduled_headway']

In [7]:
filtered_df = df[(df['delay_seconds'] >= -1200) & (df['delay_seconds'] <= 1200)]

In [8]:
df.to_csv("../datasets/MBTA-Bus-Arrival-Departure-Times_2024-12-processed.csv", index=False)